In [1]:
import pandas as pd
import numpy as np

In [13]:
#Read data, grab only the first table, not the date table
ua_file = pd.read_csv('Acura_UA_Geography.csv', header = 6, thousands=',')
n_values = np.logical_not(ua_file.isnull()).sum(axis=1)
cut_off = n_values[n_values == 0].index[0]
ua_file = ua_file[:cut_off -1]
ua_file['Users'] = ua_file['Users'].str.replace(',','')
g4_file = pd.read_csv('Acura_GA4_Geography.csv', header = 6, thousands=',')
g4_file = g4_file[['Country', 'Total users', 'Sessions']]
g4_file.rename(columns={'Total users': 'Users'}, inplace = True)

#Add data source column

ua_file['data_source'] = 'UA'
g4_file['data_source'] = 'G4'

#Append data, reset index
data = ua_file.append(g4_file).reset_index()
data.drop(columns={'index'}, inplace = True)
data

,Country,Users,Sessions,data_source
0,Canada,70685,101270.0,UA
1,United States,1315,1622.0,UA
2,Morocco,412,817.0,UA
3,India,302,731.0,UA
4,China,237,270.0,UA
...,...,...,...,...
263,Suriname,1,2.0,G4
264,Syria,1,0.0,G4
265,Uganda,1,1.0,G4
266,Vanuatu,1,1.0,G4


In [14]:
cut_off

134

In [15]:
piv = pd.pivot_table(data, values = ['Users', 'Sessions'], index = 'Country', columns = 'data_source', aggfunc = {'Users':sum, 'Sessions':sum})
piv.columns = [c[1] + ' ' +  c[0] for c in piv.columns]
piv

,G4 Sessions,UA Sessions,G4 Users,UA Users
Country,,,,
(not set),40.0,6.0,40,6
Albania,1.0,1.0,1,1
Algeria,10.0,13.0,4,4
Argentina,10.0,9.0,4,4
Armenia,1.0,1.0,1,1
...,...,...,...,...
Vanuatu,1.0,NaN,1,NaN
Venezuela,NaN,1.0,NaN,1
Vietnam,25.0,26.0,22,23


In [16]:
piv['G4 Users'] = piv['G4 Users'].astype('float')
piv['UA Users'] = piv['UA Users'].astype('float')
piv['Session Difference'] = piv['G4 Sessions'] - piv['UA Sessions']
piv['Session Variance (%)'] = (piv['G4 Sessions'] - piv['UA Sessions'])/ piv['UA Sessions']
piv['Users Difference'] = piv['G4 Users'] - piv['UA Users']
piv['Users Variance (%)'] = (piv['G4 Users'] - piv['UA Users'])/ piv['UA Users']
piv = piv[[ 'G4 Users', 'UA Users', 'Users Difference', 'Users Variance (%)', 'G4 Sessions', 'UA Sessions', 'Session Difference', 'Session Variance (%)']]
piv.sort_values(by = ['G4 Users'], ascending = False, inplace = True)

In [17]:
piv

,G4 Users,UA Users,Users Difference,Users Variance (%),G4 Sessions,UA Sessions,Session Difference,Session Variance (%)
Country,,,,,,,,
Canada,70083.0,70685.0,-602.0,-0.008517,94879.0,101270.0,-6391.0,-0.063109
United States,1732.0,1315.0,417.0,0.317110,2092.0,1622.0,470.0,0.289766
Morocco,423.0,412.0,11.0,0.026699,736.0,817.0,-81.0,-0.099143
India,297.0,302.0,-5.0,-0.016556,346.0,731.0,-385.0,-0.526676
China,243.0,237.0,6.0,0.025316,272.0,270.0,2.0,0.007407
...,...,...,...,...,...,...,...,...
Myanmar (Burma),1.0,1.0,0.0,0.000000,1.0,1.0,0.0,0.000000
Nepal,1.0,1.0,0.0,0.000000,1.0,1.0,0.0,0.000000
Nicaragua,1.0,1.0,0.0,0.000000,1.0,1.0,0.0,0.000000


In [18]:
piv.to_excel('Acura_Pivot_Geography.xlsx')